In [8]:
import os
print(os.getcwd())

c:\Users\Sures\OneDrive\Desktop\New folder\NLP\Week-2


In [17]:
import pandas as pd 

df = pd.read_csv(
    r"C:\Users\Sures\OneDrive\Desktop\New folder\NLP\Week-2\data\spam.csv",
    encoding="latin-1"
)
print(df.head())

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [ ]:
df = df[['v1', 'v2']]
df.columns = ['label', 'text']
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [31]:
# preprocessing tokenisation 
from nltk.tokenize import word_tokenize

df['tokens'] = df['text'].apply(word_tokenize)
print(df.head())

  label                                               text  \
0   ham  Go until jurong point, crazy.. Available only ...   
1   ham                      Ok lar... Joking wif u oni...   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...   
3   ham  U dun say so early hor... U c already then say...   
4   ham  Nah I don't think he goes to usf, he lives aro...   

                                              tokens  
0  [Go, until, jurong, point, ,, crazy, .., Avail...  
1           [Ok, lar, ..., Joking, wif, u, oni, ...]  
2  [Free, entry, in, 2, a, wkly, comp, to, win, F...  
3  [U, dun, say, so, early, hor, ..., U, c, alrea...  
4  [Nah, I, do, n't, think, he, goes, to, usf, ,,...  


In [33]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sures\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sures\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sures\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Sures\AppData\Roaming\nltk_data...


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

import re 

def preprocess_tokens(tokens) :
    cleaned = []
    for token in tokens:
        token = token.lower() 
        token = re.sub(r'[^a-z]', '', token) 
        
        if token == '':
            continue
        if token in stop_words:
            continue
        
        token = lemmatizer.lemmatize(token)  
        
        cleaned.append(token)
    return cleaned

df['clean_tokens'] = df['tokens'].apply(preprocess_tokens)
df.head()

,label,text,tokens,clean_tokens
0,ham,"Go until jurong point, crazy.. Available only ...","[Go, until, jurong, point, ,, crazy, .., Avail...","[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,"[Ok, lar, ..., Joking, wif, u, oni, ...]","[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F...","[free, entry, wkly, comp, win, fa, cup, final,..."
3,ham,U dun say so early hor... U c already then say...,"[U, dun, say, so, early, hor, ..., U, c, alrea...","[u, dun, say, early, hor, u, c, already, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...","[Nah, I, do, n't, think, he, goes, to, usf, ,,...","[nah, nt, think, go, usf, life, around, though]"


In [41]:
df = df.copy()
df = df[['label','clean_tokens']]
print(df.head())

  label                                       clean_tokens
0   ham  [go, jurong, point, crazy, available, bugis, n...
1   ham                     [ok, lar, joking, wif, u, oni]
2  spam  [free, entry, wkly, comp, win, fa, cup, final,...
3   ham      [u, dun, say, early, hor, u, c, already, say]
4   ham    [nah, nt, think, go, usf, life, around, though]


In [38]:
import numpy as np

In [42]:
import gensim.downloader as api 
from gensim.models import KeyedVectors
model = api.load('word2vec-google-news-300')

def convert2vec(tokens):
    valid_vectors = [model[word] for word in tokens if word in model]
    if not valid_vectors:
        return np.zeros(model.vector_size)
    return np.mean(valid_vectors, axis=0)

df['vectorized_clean_tokens'] = df['clean_tokens'].apply(convert2vec)
print(df['vectorized_clean_tokens'].shape)

(5572,)


In [48]:
from sklearn.model_selection import train_test_split

X = df['vectorized_clean_tokens']
y = df['label']
X_train,X_test,y_train,y_test = train_test_split(
    X,
    y,
    test_size = 0.2,
    random_state = 42,
    stratify=y
)

In [50]:
import numpy as np

X_train = np.vstack(X_train.to_numpy())
X_test  = np.vstack(X_test.to_numpy())

In [54]:
type(X_train)
X_train.shape

(4457, 300)

In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,recall_score,f1_score

new_model = LogisticRegression(
        max_iter = 1000,
        class_weight="balanced"
    ).fit(X_train,y_train)

y_pred = new_model.predict(X_test)

# Metrics
precision = precision_score(y_test, y_pred, pos_label="spam")
recall = recall_score(y_test, y_pred, pos_label="spam")
f1 = f1_score(y_test, y_pred, pos_label="spam")

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.695
Recall: 0.9328859060402684
F1-score: 0.7965616045845272


In [56]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[905  61]
 [ 10 139]]
              precision    recall  f1-score   support

         ham       0.99      0.94      0.96       966
        spam       0.69      0.93      0.80       149

    accuracy                           0.94      1115
   macro avg       0.84      0.93      0.88      1115
weighted avg       0.95      0.94      0.94      1115

